![alt text](../img/header.jpg)

# Density Dependent Flow and Transport

In [ ]:
import os
import sys
import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy

import config

print(sys.version)
print('python executable: {}'.format(sys.executable))
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

In [ ]:
os.path.isfile(config.mf6betaexe)
errmsg = ('could not find mf6beta. '
          'download from here: '
          'https://github.com/langevin-usgs/mf6flopy2019_classrepo/releases')
assert os.path.isfile(config.mf6betaexe), errmsg

In [ ]:
# Be careful here!!
#flopy.mf6.utils.generate_classes(branch='mf6beta')

In [ ]:
def get_model(workspace, model_name='mymodel'):

    lx = 2.
    lz = 1.

    nlay = 20
    nrow = 1
    ncol = 40
    nper = 1
    delr = lx / ncol
    delc = 1.
    top = 1.
    delz = lz / nlay
    botm = list(top - np.arange(delz, nlay * delz + delz, delz))

    perlen = [0.5]
    nstp = [500]
    tsmult = [1.]
    steady = [True]
    tdis_rc = []
    for i in range(nper):
        tdis_rc.append((perlen[i], nstp[i], tsmult[i]))

    # This should be changed to one true and one false after scott fixes
    # flopy.
    single_matrix = False
    nouter, ninner = 100, 300
    hclose, rclose, relax = 1e-10, 1e-6, 0.97

    name = model_name

    # build MODFLOW 6 files
    ws = workspace
    sim = flopy.mf6.MFSimulation(sim_name=name, version='mf6',
                                 exe_name=config.mf6betaexe,
                                 sim_ws=ws)
    # create tdis package
    tdis = flopy.mf6.ModflowTdis(sim, time_units='DAYS',
                                 nper=nper, perioddata=tdis_rc)

    # create gwf model
    gwfname = 'gwf_' + name
    gwtname = 'gwt_' + name

    gwf = flopy.mf6.MFModel(sim, model_type='gwf6', modelname=gwfname,
                            model_nam_file='{}.nam'.format(gwfname))
    gwf.name_file.save_flows = True

    imsgwf = flopy.mf6.ModflowIms(sim, print_option='ALL',
                                  outer_hclose=hclose,
                                  outer_maximum=nouter,
                                  under_relaxation='NONE',
                                  inner_maximum=ninner,
                                  inner_hclose=hclose, rcloserecord=rclose,
                                  linear_acceleration='BICGSTAB',
                                  scaling_method='NONE',
                                  reordering_method='NONE',
                                  relaxation_factor=relax,
                                  filename='{}.ims'.format(gwfname))
    if single_matrix:
        sim.register_ims_package(imsgwf, [gwfname, gwtname])
    else:
        sim.register_ims_package(imsgwf, [gwfname])

    dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                                  delr=delr, delc=delc,
                                  top=top, botm=botm)

    # initial conditions
    ic = flopy.mf6.ModflowGwfic(gwf, strt=1.)

    # node property flow
    npf = flopy.mf6.ModflowGwfnpf(gwf, xt3doptions=False,
                                  save_flows=True,
                                  save_specific_discharge=True,
                                  icelltype=0,
                                  k=864.)

    buy = flopy.mf6.ModflowGwfbuy(gwf)

    def chd_value(k):
        depth = k * delz + 0.5 * delz
        hf = top + 0.025 * depth
        return hf

    # chd files
    chdlist1 = []
    for k in range(nlay):
        chdlist1.append([(k, 0, ncol - 1), chd_value(k), 35.])
    chd1 = flopy.mf6.ModflowGwfchd(gwf,
                                   stress_period_data=chdlist1,
                                   print_input=True,
                                   print_flows=True,
                                   save_flows=False,
                                   pname='CHD-1',
                                   auxiliary='CONCENTRATION',
                                   filename='{}.chd'.format(gwfname))

    wellist1 = []
    qwell = 5.7024 / nlay
    for k in range(nlay):
        wellist1.append([(k, 0, 0), qwell, 0.])
    wel1 = flopy.mf6.ModflowGwfwel(gwf,
                                   stress_period_data=wellist1,
                                   print_input=True,
                                   print_flows=True,
                                   save_flows=False,
                                   pname='WEL-1',
                                   auxiliary='CONCENTRATION',
                                   filename='{}.wel'.format(gwfname))

    # output control
    oc = flopy.mf6.ModflowGwfoc(gwf,
                                budget_filerecord='{}.cbc'.format(gwfname),
                                head_filerecord='{}.hds'.format(gwfname),
                                headprintrecord=[
                                    ('COLUMNS', 10, 'WIDTH', 15,
                                     'DIGITS', 6, 'GENERAL')],
                                saverecord=[('HEAD', 'ALL'),
                                            ('BUDGET', 'ALL')],
                                printrecord=[('HEAD', 'LAST'),
                                             ('BUDGET', 'LAST')])

    # create gwt model
    gwt = flopy.mf6.MFModel(sim, model_type='gwt6', modelname=gwtname,
                            model_nam_file='{}.nam'.format(gwtname))

    if not single_matrix:
        imsgwt = flopy.mf6.ModflowIms(sim, print_option='ALL',
                                      outer_hclose=hclose,
                                      outer_maximum=nouter,
                                      under_relaxation='NONE',
                                      inner_maximum=ninner,
                                      inner_hclose=hclose, rcloserecord=rclose,
                                      linear_acceleration='BICGSTAB',
                                      scaling_method='NONE',
                                      reordering_method='NONE',
                                      relaxation_factor=relax,
                                      filename='{}.ims'.format(gwtname))
        sim.register_ims_package(imsgwt, [gwt.name])

    dis = flopy.mf6.ModflowGwtdis(gwt, nlay=nlay, nrow=nrow, ncol=ncol,
                                  delr=delr, delc=delc,
                                  top=top, botm=botm)

    # initial conditions
    ic = flopy.mf6.ModflowGwtic(gwt, strt=35.,
                                filename='{}.ic'.format(gwtname))

    # advection
    adv = flopy.mf6.ModflowGwtadv(gwt, scheme='UPSTREAM',
                                  filename='{}.adv'.format(gwtname))

    # dispersion
    diffc = 0.57024
    dsp = flopy.mf6.ModflowGwtdsp(gwt, xt3d=False, diffc=diffc,
                                  # alh=0., alv=0., ath=0., atv=0.,
                                  filename='{}.dsp'.format(gwtname))

    # storage
    porosity = 0.35
    sto = flopy.mf6.ModflowGwtsto(gwt, porosity=porosity,
                                  filename='{}.sto'.format(gwtname))

    # sources
    sourcerecarray = [('CHD-1', 1, 'CONCENTRATION'),
                      ('WEL-1', 1, 'CONCENTRATION')]
    ssm = flopy.mf6.ModflowGwtssm(gwt, sources=sourcerecarray,
                                  filename='{}.ssm'.format(gwtname))

    # output control
    oc = flopy.mf6.ModflowGwtoc(gwt,
                                budget_filerecord='{}.cbc'.format(gwtname),
                                concentration_filerecord='{}.ucn'.format(
                                    gwtname),
                                concentrationprintrecord=[
                                    ('COLUMNS', 10, 'WIDTH', 15,
                                     'DIGITS', 6, 'GENERAL')],
                                saverecord=[('CONCENTRATION', 'ALL')],
                                printrecord=[('CONCENTRATION', 'LAST'),
                                             ('BUDGET', 'LAST')])

    # GWF GWT exchange
    gwfgwt = flopy.mf6.ModflowGwfgwt(sim, exgtype='GWF6-GWT6',
                                     exgmnamea=gwfname, exgmnameb=gwtname,
                                     filename='{}.gwfgwt'.format(name))

    return sim




In [ ]:
workspace = 'ex0x-completed-density'
sim = get_model(workspace)

In [ ]:
# write the input files
sim.write_simulation()

In [ ]:
# run the simulation
sim.run_simulation()

In [ ]:
print(sim.model_names)
gwfname = sim.model_names[0]
gwtname = sim.model_names[1]
gwf = sim.get_model(gwfname)
gwt = sim.get_model(gwtname)

In [ ]:
fname = os.path.join(workspace, gwtname + '.ucn')
concobj = flopy.utils.HeadFile(fname, text='CONCENTRATION')
conc = concobj.get_data()

fname = os.path.join(workspace, gwfname + '.cbc')
bobj = flopy.utils.CellBudgetFile(fname, precision='double')
spdis = bobj.get_data(text='DATA-SPDIS')

In [ ]:
fig = plt.figure(figsize=(15, 10))
ax = plt.subplot(1, 1, 1, aspect='equal')

pmv = flopy.plot.PlotCrossSection(model=gwf, line={'row': 0})
pmv.plot_grid(color='.5')
pmv.plot_specific_discharge(spdis, color='0.5')
pmv.contour_array(conc, levels=np.linspace(0, 35, 10));